In [ ]:
import pandas as pd
import numpy as np

from jqdatasdk import *

# User Authentication:
a = auth('JointQuant_UserName', 'JoinQuant_Password')

count=get_query_count()
print(count)

auth success 
{'total': 10000000, 'spare': 9000000}


In [ ]:
# Define and speficy the start_date & end_date of the data
# Note: Define range of dates in minutes in seperately
# Since the data of transaction in Minutes and Hours are numerous and much larger than Daily's.
start_date_normal = '2000-01-01'
end_date_normal = '2021-12-05'
start_date_InMinutes = '2021-01-01'
end_date_InMinutes = '2021-10-20'

In [ ]:
# Get all available dates of transaction since the start_date to end_date that specified.
# Example: 2021-09-20 to 2021-10-20 
trade_days = get_trade_days(start_date=start_date_normal, end_date=end_date_normal)
print(trade_days)

[datetime.date(2005, 1, 4) datetime.date(2005, 1, 5)
 datetime.date(2005, 1, 6) ... datetime.date(2021, 12, 1)
 datetime.date(2021, 12, 2) datetime.date(2021, 12, 3)]


In [ ]:
# Get the historical data of "Ping An Bank" starts with encode of SZSE (Shenzhen Stock Exchange)
normalized = normalize_code(['000001'])
print(normalized)
Pingan = get_security_info(normalized[0])
print(Pingan)

['000001.XSHE']
000001.XSHE


In [ ]:
# Transaction in Daily
stocklist = Pingan
price_table_daily = get_price(security=stocklist, start_date=trade_days[0], end_date=trade_days[len(trade_days)-1],
                  frequency='daily', fields=None, skip_paused=False, fq='pre', panel=False)

In [ ]:
print(price_table_daily.head(),type(price_table_daily))

# Export to csv file
price_table_daily.to_excel('Price_Of_PingAn_daily.xlsx', encoding='utf-8', sheet_name='平安银行')

            open  close  high   low      volume       money
2005-01-04  1.47   1.46  1.47  1.45   7867519.0  11465603.0
2005-01-05  1.46   1.45  1.47  1.42  14396761.0  20718559.0
2005-01-06  1.45   1.46  1.47  1.44  11913717.0  17333840.0
2005-01-07  1.47   1.46  1.48  1.45   8427452.0  12302853.0
2005-01-10  1.46   1.47  1.47  1.43  11760203.0  17111498.0 <class 'pandas.core.frame.DataFrame'>


In [ ]:
# In-case: Transaction in minutes
# Since start_date to end_date of all available dates of transaction
# Example: 2021-09-20 00:00 to 2021-10-20 23:59
trade_days_PerMinutes = get_trade_days(start_date=start_date_InMinutes , end_date=end_date_InMinutes )
price_table_minutes = get_price(security=stocklist, start_date=trade_days_PerMinutes[0], 
                              end_date=trade_days_PerMinutes[len(trade_days)-1],
                              frequency='1m', fields=None, skip_paused=False, fq='pre', panel=False)

# In-case: Transaction in Hours
# Since start_date to end_date of all available dates of transaction
# Example: 2021-09-20 12 AM to 2021-10-20 11 PM

price_table_hours = get_price(security=stocklist, start_date=trade_days[0], end_date=trade_days[len(trade_days)-1],
                  frequency='60m', fields=None, skip_paused=False, fq='pre', panel=False)
price_table_minutes = get_price(security=stocklist, start_date=trade_days[0], end_date=trade_days[len(trade_days)-1],
                  frequency='1m', fields=None, skip_paused=False, fq='pre', panel=False)

print(price_table_minutes)
price_table_daily.to_csv('Price_Of_SingleStock_Minutes.csv', encoding='utf-8')
print(price_table_hours)
price_table_daily.to_csv('Price_Of_SingleStock_hours.csv', encoding='utf-8')

In [ ]:
# Get the value of all kinds of factors by group of category in Factor Libary.
factorlist = get_all_factors()
print(factorlist.head()) 

In [ ]:
category_info = list(factorlist.loc[:,'category'])
category_info_CHI = list(factorlist.loc[:,'category_intro'])
category_list = []
category_list_CHI = []
for i in range(len(category_info)):
    if category_info[i] != category_info[i-1] and category_info[i] != 'pershare' and category_info[i] != 'quality':
        category_list.append(category_info[i])
        category_list_CHI.append(category_info_CHI[i])
        
print(f"List of category of factors of dataset: ")
for j in range(len(category_list)):
    print(category_list[j], category_list_CHI[j])

List of category of factors of dataset: 
basics 基础科目及衍生类因子
emotion 情绪类因子
growth 成长类因子
momentum 动量类因子
risk 风险类因子
style 风险因子 - 风格因子
technical 技术指标因子


In [ ]:
import sys
for t_factor in category_list:
    factor_category = factorlist.loc[factorlist['category'] == t_factor]
    factorName = factor_category.iloc[1:,0].tolist()
    factorName_Chinese = factor_category.iloc[1:,1].tolist()
    print(factorName[0:5])
    # Obtain the table of values in current categories of factor libary 
    # For instances: 
    #      Category: basics.
    #      Values:   
    factor_values = get_factor_values(securities=stocklist, factors=factorName, \
                                            start_date=trade_days[0], end_date=trade_days[len(trade_days)-1])
    print(len(factor_values))
    # Obtain the table of values(factor information) in current categories into the excel with pair time series
    df = pd.DataFrame(factor_values[factorName[0]])
    factor_data = pd.DataFrame();
    for i in range(0, len(factorName)):
        df = pd.DataFrame(factor_values[factorName[i]])
        current_value = df.values
        new_df = pd.DataFrame(current_value, columns=[factorName[i]])
        factor_data = pd.concat([factor_data, new_df],axis=1)
        
    # Export the table of values(factor information) in current categories to the excel in names with reasonable 
    factor_data.index = df.index
    factor_data = factor_data.fillna(value=0)
    print(factor_data.head())
    factor_data.to_excel(f"factors_{t_factor}.xlsx",encoding='utf-8')

['asset_impairment_loss_ttm', 'cash_flow_to_price_ratio', 'circulating_market_cap', 'EBIT', 'EBITDA']
36
            asset_impairment_loss_ttm  cash_flow_to_price_ratio  \
2005-01-04               1.643679e+09                 -0.332060   
2005-01-05               1.643679e+09                 -0.335143   
2005-01-06               1.643679e+09                 -0.332060   
2005-01-07               1.643679e+09                 -0.332568   
2005-01-10               1.643679e+09                 -0.328537   

            circulating_market_cap         EBIT       EBITDA  \
2005-01-04            9.189040e+09  146563918.0  742058579.0   
2005-01-05            9.104480e+09  146563918.0  742058579.0   
2005-01-06            9.189040e+09  146563918.0  742058579.0   
2005-01-07            9.174950e+09  146563918.0  742058579.0   
2005-01-10            9.287700e+09  146563918.0  742058579.0   

            financial_assets  financial_expense_ttm  financial_liability  \
2005-01-04      5.889581e+09   

['Kurtosis20', 'Kurtosis60', 'sharpe_ratio_120', 'sharpe_ratio_20', 'sharpe_ratio_60']
11
            Kurtosis20  Kurtosis60  sharpe_ratio_120  sharpe_ratio_20  \
2005-01-04    0.000000    0.000000               0.0              0.0   
2005-01-05    0.000000    0.000000               0.0              0.0   
2005-01-06    0.000000    0.000000               0.0              0.0   
2005-01-07    0.000000    0.000000               0.0              0.0   
2005-01-10   -2.884535   -2.884535               0.0              0.0   

            sharpe_ratio_60  Skewness120  Skewness20  Skewness60  Variance120  \
2005-01-04              0.0     0.000000    0.000000    0.000000     0.000000   
2005-01-05              0.0     0.000000    0.000000    0.000000     0.000000   
2005-01-06              0.0     0.000000    0.000000    0.000000     0.043024   
2005-01-07              0.0     0.496860    0.496860    0.496860     0.021716   
2005-01-10              0.0    -0.422431   -0.422431   -0.422431  

In [ ]:
# Get current category of factors within factor libary in separate 
# For instances: 
#      Category: basics.
#      Values:  ...

basics_factor = factorlist.loc[factorlist['category_intro'] == '基础科目及衍生类因子']
risk_factor = factorlist.loc[factorlist['category'] == 'risk']
technical_factor = factorlist.loc[factorlist['category'] == 'technical']

print(basics_factor.head())
print(risk_factor.head())
print(technical_factor.head())

In [ ]:
# Obtain the table of values in category of "Basic" within factor libary 
    #      Category: basics.
    #      Numbers: 36
    #      Valuess: 'asset_impairment_loss_ttm', 'cash_flow_to_price_ratio', 'circulating_market_cap', 'EBIT', 'EBITDA'
factor_name = basics_factor.iloc[1:,0].tolist()
factor_name_chinese = basics_factor.iloc[1:,1].tolist()
print(factor_name[0:5])
print(factor_name_chinese[0:5])

['asset_impairment_loss_ttm', 'cash_flow_to_price_ratio', 'circulating_market_cap', 'EBIT', 'EBITDA']
['资产减值损失TTM', '现金流市值比', '流通市值', '息税前利润', '息税折旧摊销前利润']


In [ ]:
# Obtain the table of values(factor information) in "basic" categories into the excel with pair time series
factor_values_basic = get_factor_values(securities=stocklist, factors=factor_name
                                 ,start_date=trade_days[0], end_date=trade_days[len(trade_days)-1])

In [ ]:
# Export the table of values(factor information) in "basic" categories onto the excel in names with reasonable 
df = pd.DataFrame(factor_values_basic[factor_name[0]])
factor_data = pd.DataFrame();

for i in range(0, len(factor_name)):
    df = pd.DataFrame(factor_values_basic[factor_name[i]])
    current_value = df.values
    new_df = pd.DataFrame(current_value, columns=[factor_name[i]])
    factor_data = pd.concat([factor_data,new_df],axis=1)
    
factor_data.index = df.index
factor_data = factor_data.fillna(value=0)
print(factor_data.head())
factor_data.to_excel('./factors_basic.xlsx',encoding='utf-8')

Empty DataFrame
Columns: [asset_impairment_loss_ttm, cash_flow_to_price_ratio, circulating_market_cap, EBIT, EBITDA, financial_assets, financial_expense_ttm, financial_liability, goods_sale_and_service_render_cash_ttm, gross_profit_ttm, interest_carry_current_liability, interest_free_current_liability, market_cap, net_debt, net_finance_cash_flow_ttm, net_interest_expense, net_invest_cash_flow_ttm, net_operate_cash_flow_ttm, net_profit_ttm, net_working_capital, non_operating_net_profit_ttm, non_recurring_gain_loss, np_parent_company_owners_ttm, OperateNetIncome, operating_assets, operating_cost_ttm, operating_liability, operating_profit_ttm, operating_revenue_ttm, retained_earnings, sales_to_price_ratio, sale_expense_ttm, total_operating_cost_ttm, total_operating_revenue_ttm, total_profit_ttm, value_change_profit_ttm]
Index: []

[0 rows x 36 columns]


In [ ]:
# Obtain the table of values in category of "Basic" within factor libary 
    #      Category: risks.
    #      Numbers: 11
    #      Values: 'Kurtosis20', 'Kurtosis60', 'sharpe_ratio_120', 'sharpe_ratio_20', 'sharpe_ratio_60'
factor_name_risk = risk_factor.iloc[1:,0].tolist()
factor_name_CHIrisk = risk_factor.iloc[1:,1].tolist()
print(factor_name_risk[0:5])
print(factor_name_CHIrisk[0:5])
# Obtain the table of values(factor information) in "risk" categories into the excel with pair time series
factor_values_risk = get_factor_values(securities=stocklist, factors=factor_name_risk,\
                                  start_date=trade_days[0], end_date=trade_days[len(trade_days)-1])

['Kurtosis20', 'Kurtosis60', 'sharpe_ratio_120', 'sharpe_ratio_20', 'sharpe_ratio_60']
['个股收益的20日峰度', '个股收益的60日峰度', '120日夏普比率', '20日夏普比率', '60日夏普比率']


In [ ]:
# Export the table of values(factor information) in "risk" categories onto the excel in names with reasonable 
df = pd.DataFrame(factor_values_risk[factor_name_risk[0]])
factor_data_risk = pd.DataFrame();

for i in range(0, len(factor_name_risk)):
    df = pd.DataFrame(factor_values[factor_name_risk[i]])
    current_value = df.values
    new_df = pd.DataFrame(current_value, columns=[factor_name_risk[i]])
    factor_data_risk = pd.concat([factor_data_risk, new_df], axis=1)
    
factor_data_risk.index = df.index
new_factor_data_risk = factor_data_risk.dropna()
print(new_factor_data_risk.head())
new_factor_data_risk.to_excel('./factors_risk.xlsx',encoding='utf-8')

            Kurtosis20  Kurtosis60  sharpe_ratio_120  sharpe_ratio_20  \
2014-01-02   -0.873462   -0.807671               0.0         0.732323   
2014-01-03   -0.946424   -0.797069               0.0        -0.452112   
2014-01-06   -1.142635   -0.872292               0.0        -1.161135   
2014-01-07   -1.127541   -0.856073               0.0        -1.269184   
2014-01-08   -0.822797   -0.804987               0.0        -1.764395   

            sharpe_ratio_60  Skewness120  Skewness20  Skewness60  Variance120  \
2014-01-02         1.306094     0.664376   -0.083183   -0.044930     0.141487   
2014-01-03         0.604256     0.687482    0.104498    0.022838     0.140829   
2014-01-06         0.191912     0.687300    0.156118    0.026210     0.142565   
2014-01-07         0.007157     0.700211    0.193550    0.058387     0.140613   
2014-01-08         0.267705     0.707778    0.300828    0.026670     0.138642   

            Variance20  Variance60  
2014-01-02    0.114516    0.102915  


In [ ]:
# Obtain the table of values in category of "Technical" within factor libary 
    #      Category: Technical.
    #      Numbers: 15
    #      Values: 'boll_up', 'EMA5', 'EMAC10', 'EMAC12', 'EMAC120'
factor_name_tech = technical_factor.iloc[1:,0].tolist()
factor_name_CHItech = technical_factor.iloc[1:,1].tolist()
print(factor_name_tech[0:5])
print(factor_name_CHItech[0:5])
# Obtain the table of values(factor information) in "Technical" categories into the excel with pair time series
factor_values_tech = get_factor_values(securities=stocklist, factors=factor_name_tech,\
                                  start_date=trade_days[0], end_date=trade_days[len(trade_days)-1])

['boll_up', 'EMA5', 'EMAC10', 'EMAC12', 'EMAC120']
['上轨线（布林线）指标', '5日指数移动均线', '10日指数移动均线', '12日指数移动均线', '120日指数移动均线']


In [ ]:
# Export the table of values(factor information) in "Technical" categories onto the excel in names with reasonable 
df = pd.DataFrame(factor_values_tech[factor_name_tech[0]])
factor_data_tech = pd.DataFrame();

for i in range(0, len(factor_name_tech)):
    df = pd.DataFrame(factor_values_tech[factor_name_tech[i]])
    current_value = df.values
    new_df = pd.DataFrame(current_value, columns=[factor_name_tech[i]])
    factor_data_tech = pd.concat([factor_data_tech, new_df], axis=1)
    
factor_data_tech.index = df.index
new_factor_dataTech = factor_data_tech.dropna()
print(new_factor_dataTech.head())
new_factor_dataTech.to_excel('./factors_technical.xlsx',encoding='utf-8')

             boll_up      EMA5    EMAC10    EMAC12   EMAC120    EMAC20  \
2005-01-04  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2005-01-05  1.009315  1.004444  1.005455  1.005641  1.006556  1.006032   
2005-01-06  1.002631  0.998528  0.999015  0.999138  0.999892  0.999429   
2005-01-07  1.002631  0.999019  0.999194  0.999271  0.999894  0.999484   
2005-01-10  0.993927  0.990640  0.988654  0.988330  0.987041  0.987712   

              EMAC26     MAC10    MAC120     MAC20      MAC5     MAC60  \
2005-01-04  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2005-01-05  1.006173  1.006000  1.006611  1.006333  1.005333  1.006556   
2005-01-06  0.999546  0.999338  0.999945  0.999669  0.998675  0.999890   
2005-01-07  0.999579  0.999338  0.999945  0.999669  0.998675  0.999890   
2005-01-10  0.987512  0.987582  0.986983  0.987255  0.988235  0.987037   

               MACDC      MFI14  price_no_fq  
2005-01-04  0.000000   0.000000         6.52  
2005-01-05 -0.00

In [ ]:
logout()

已退出
